In [1]:
import pandas as pd
# import numpy as np
import datetime
# from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = 'df_close_clean'

str_today = datetime.date.today().strftime('%Y-%m-%d')
###########################################################################################################
fp_df_eval_sym_freq_results_train    = f'df_eval_sym_freq_results_train_{str_today}'
fp_df_eval_sym_freq_results_validate = f'df_eval_sym_freq_results_validate_{str_today}'
fp_df_eval_sym_freq_results_test     = f'df_eval_sym_freq_results_test_{str_today}'
###########################################################################################################

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

#### Check run results

In [2]:
# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_train)
# df
# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
# df
# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_test)
# df

#### Create column names

In [3]:
col_add0 = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt']
col_add1 = ['grp(CAGR)_mean',         'grp(CAGR)_std',         'grp(CAGR)_mean/std']
col_add2 = ['grp(CAGR/UI)_mean',      'grp(CAGR/UI)_std',      'grp(CAGR/UI)_mean/std']
col_add3 = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
col_add4 = ['grp(retnStd/UI)_mean',   'grp(retnStd/UI)_std',   'grp(retnStd/UI)_mean/std']
col_add5 = ['SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI']

col_add_total = col_add0 + col_add1 + col_add2 + col_add3 + col_add4 + col_add5
print(f'col_add_total:\n{col_add_total}, total columns: {len(col_add_total)}')

col_add_total:
['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt', 'grp(CAGR)_mean', 'grp(CAGR)_std', 'grp(CAGR)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI'], total columns: 23


#### DANGER: Create a new Dataframe to store run results. Previous data will be destroyed. To create a new dataframe, uncomment one choice from below.

In [4]:
# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_train)
# df

# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_validate)
# df

# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_test)
# df

#### Select run parameters. Parameters can also be passed using papermill by running yf_7_freq_cnt_pm_.ipynb

In [5]:
# SELECT RUN PARAMETERS.async Parameters can also be passed using papermill by running yf_7_freq_cnt_pm_.ipynb
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# select run type
run_type = 'train'
# run_type = 'validate'
# run_type = 'test'

# number of max lookback tuples to create for iloc iloc_start_train:iloc_end_train:iloc_end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
# n_samples = 400  
n_samples = 2

# for training, the number of days to lookback from iloc max-lookback iloc_end_train
days_lookbacks = [5]
days_lookbacks.sort()

# number of days from iloc_end_train are used to evaluate effectiveness of the training
days_eval = 5

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  #  start index of n_top_syms for evaluation
syms_end = 10  #  end index of n_top_syms for evaluation

In [6]:
# Parameters
verbose = True
store_results = True
run_type = "test"
n_samples = 57
n_top_syms = 20
syms_start = 0
syms_end = 10
days_eval = 6
days_lookbacks = [15, 30, 60, 120]


#### Split dataframe into Train, Validate and Test

In [7]:
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)

#### Load previous data according to run_type

In [8]:
if run_type == 'train':
  df_eval_sym_freq_results_train = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_train)
  df = df_train.copy()    
elif run_type == 'validate':
  df_eval_sym_freq_results_validate = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
  df = df_val.copy()
elif run_type == 'test':
  df_eval_sym_freq_results_test = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_test)
  df = df_test.copy()
else:
  msg_stop = f"ERROR run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
  raise SystemExit(msg_stop )  

#### Print lengths of dataframes

In [9]:
len_df = len(df)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f'run_type: {run_type}, len(df): {len(df)}')
print(f'len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test} ')

run_type: test, len(df): 150
len_df_train: 1050, len_df_val: 300, len_df_test: 150 


#### Create a sets of iloc lookback slices (iloc_start_train:iloc_end_train:iloc_end_eval), where:  
* iloc_end_train - iloc_start_train = days_lookback
* iloc_end_eval - iloc_end_train = days_eval  
#### for example, if given:  
* n_samples = 2  
* days_lookbacks = [10, 20, 30]  
* days_eval = 5  
#### a possible result is:  
  - max_lookback_slices: [(417, 447, 452), (265, 295, 300)], where:
    - len(max_lookback_slices) = n_samples = 2
    - middle number in the tuples, 447 and 295, is the iloc of the "pivot day" for the days in "days_lookbacks" to lookback
    - 447 - 417 = middle number - first number = max(days_lookbacks) = 30
    - 295 - 265 = middle number - first number = max(days_lookbacks) = 30      
    - 452 - 447 = last number - middle number = days_eval = 5
    - 300 - 295 = last number - middle number = days_eval = 5    
  - sets_lookback_slices: [[(437, 447, 452), (427, 447, 452), (417, 447, 452)], [(285, 295, 300), (275, 295, 300), (265, 295, 300)]], where:
    - len(sets_lookback_slices) = n_samples = 2
    - last tuple in each list, i.e. (417, 447, 452) and (265, 295, 300), is a tuple from max_lookback_slices
    - where a set, e.g. [(437, 447, 452), (427, 447, 452), (417, 447, 452)]:
      - middle number, 447, iloc of the "pivot day" is constant for that set
      - middle number - first number, is the training period specified in days_lookbacks
        - 447 - 437 = middle number - first number = days_lookbacks[0] = 10
        - 447 - 427 = middle number - first number = days_lookbacks[1] = 20
        - 447 - 417 = middle number - first number = days_lookbacks[2] = 30
      - last number, 452, iloc of the end of the evaluation period is constant
        - 452 - 447 = last number - middle number = days_eval = 5

In [10]:
# return n_samples slices
max_lookback_slices = _3_random_slices(len_df, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'number of max_lookback_slices is equal to n_samples = {n_samples}')
  print(f'max_lookback_slices:\n{max_lookback_slices}\n')
  print(f'number of sets in sets_lookback_slices is equal to n_samples = {n_samples}')  
  print(f'sets_lookback_slices:\n{sets_lookback_slices}\n')
  print(f'number of tuples in each "set of lookback slices" is equal to len(days_lookbacks): len({days_lookbacks})')    

number of max_lookback_slices is equal to n_samples = 57
max_lookback_slices:
[(15, 135, 141), (15, 135, 141), (23, 143, 149), (12, 132, 138), (12, 132, 138), (22, 142, 148), (11, 131, 137), (17, 137, 143), (9, 129, 135), (17, 137, 143), (12, 132, 138), (19, 139, 145), (16, 136, 142), (1, 121, 127), (19, 139, 145), (8, 128, 134), (17, 137, 143), (17, 137, 143), (1, 121, 127), (21, 141, 147), (5, 125, 131), (1, 121, 127), (11, 131, 137), (23, 143, 149), (21, 141, 147), (2, 122, 128), (13, 133, 139), (12, 132, 138), (21, 141, 147), (5, 125, 131), (11, 131, 137), (17, 137, 143), (20, 140, 146), (22, 142, 148), (17, 137, 143), (20, 140, 146), (10, 130, 136), (9, 129, 135), (3, 123, 129), (11, 131, 137), (7, 127, 133), (17, 137, 143), (17, 137, 143), (15, 135, 141), (5, 125, 131), (0, 120, 126), (4, 124, 130), (5, 125, 131), (16, 136, 142), (16, 136, 142), (0, 120, 126), (19, 139, 145), (18, 138, 144), (7, 127, 133), (0, 120, 126), (10, 130, 136), (16, 136, 142)]

number of sets in sets_loo

#### grp_top_set_syms_n_freq is a list of sub-lists, e.g.:
 - [[('AGY', 7), ('PCG', 7), ('KDN', 6), ..., ('CYT', 3)], ..., [('FCN', 9), ('HIG', 9), ('SJR', 8), ..., ('BFH', 2)]]
#### There are n_samples sub-lists. Each sub-list has n_top_syms tuples of (symbol, frequency) pairs, and is sorted in descending order of frequency. The frequency is the number of times the symbol appears in the top n_top_syms performance rankings of CAGR/UI, CAGR/retnStd and retnStd/UI. Therefore, the symbols in the sub-list are the best performing symbols for the periods in days_lookbacks.

In [11]:
# grp_top_set_syms_n_freq is a list of lists of top_set_syms_n_freq, e.g.
#   [[('AGY', 7), ('PCG', 7), ('KDN', 6), ..., ('CYT', 3)],
#    [('FCN', 9), ('HIG', 9), ('SJR', 8), ..., ('BFH', 2)]]
#   where each list is the best performing symbols from a lb_slices, e.g.
#     [(483, 513, 523), (453, 513, 523), (393, 513, 523)]  
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # grp_top_set_syms_n_freq without the frequency count

# lb_slices, e.g  [(483, 513, 523), (453, 513, 523), (393, 513, 523)],
#  is one max_lookback_slice, e.g. (393, 513, 523), along with
#  the remaining slices of the days_lookbacks, e.g. (483, 513, 523), (453, 513, 523)  
for i, lb_slices in enumerate(sets_lookback_slices):
  print(f'\n########## {i + 1} of {len(sets_lookback_slices)} lb_slices in sets_lookcak_slices ##########')
  # unsorted list of the most frequent symbols in performance metrics of the lb_slices  
  grp_most_freq_syms = []
  for j, lb_slice in enumerate(lb_slices):  # lb_slice, e.g. (246, 276, 286)
    iloc_start_train = lb_slice[0]     # iloc of start of training period
    iloc_end_train   = lb_slice[1]     # iloc of end of training period
    iloc_start_eval  = iloc_end_train  # iloc of start of evaluation period
    iloc_end_eval    = lb_slice[2]     # iloc of end of evaluation period
    lookback         = iloc_end_train - iloc_start_train
    d_eval           = iloc_end_eval - iloc_start_eval

    _df = df.iloc[iloc_start_train:iloc_end_train]
    date_iloc_start_train = _df.index[0].strftime('%Y-%m-%d')
    date_iloc_end_train = _df.index[-1].strftime('%Y-%m-%d')

    if verbose:
      print(f'days lookback:         {lookback},  {j + 1} of {len(days_lookbacks)} days_lookbacks: {days_lookbacks}')
      print(f'lb_slices:             {lb_slices}')
      print(f'lb_slice:              {lb_slice}')
      print(f'days eval:             {d_eval}')    
      print(f'iloc_iloc_start_train: {iloc_start_train}')
      print(f'iloc_iloc_end_train:   {iloc_end_train}')
      print(f'date_iloc_start_train: {date_iloc_start_train}')
      print(f'date_iloc_end_train:   {date_iloc_end_train}')


    perf_ranks, most_freq_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    # unsorted list of the most frequent symbols in performance metrics of the lb_slices  
    grp_most_freq_syms.append(most_freq_syms)  
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_freq_syms: {most_freq_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_freq_syms: {grp_most_freq_syms}')
    # grp_most_freq_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_freq_syms = [val for sublist in grp_most_freq_syms for val in sublist]
  # return "symbol, frequency" pairs of the most frequent symbols, i.e. best performing symbols,
  #  in flat_grp_most_freq_syms. The paris are sorted in descending frequency.   
  set_most_freq_syms = _6_grp_tuples_sort_sum(flat_grp_most_freq_syms, reverse=True)
  # get the top n_top_syms of the most frequent "symbol, frequency" pairs
  top_set_syms_n_freq = set_most_freq_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq, i[0] = symbol, i[1]=symbol's frequency count
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  

  # grp_top_set_syms_n_freq is a list of lists of top_set_syms_n_freq, e.g.
  #   [[('AGY', 7), ('PCG', 7), ('KDN', 6), ..., ('CYT', 3)],
  #    [('FCN', 9), ('HIG', 9), ('SJR', 8), ..., ('BFH', 2)]]
  #   where each list is the best performing symbols from a lb_slices, e.g.
  #     [(483, 513, 523), (453, 513, 523), (393, 513, 523)]    
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')


########## 1 of 57 lb_slices in sets_lookcak_slices ##########
days lookback:         15,  1 of 4 days_lookbacks: [15, 30, 60, 120]
lb_slices:             [(120, 135, 141), (105, 135, 141), (75, 135, 141), (15, 135, 141)]
lb_slice:              (120, 135, 141)
days eval:             6
iloc_iloc_start_train: 120
iloc_iloc_end_train:   135
date_iloc_start_train: 2023-01-25
date_iloc_end_train:   2023-02-14


perf_ranks: {'period-15': {'r_CAGR/UI': array(['CTLT', 'TSLA', 'STNG', 'TNK', 'ELF', 'ASC', 'FCF', 'TNP', 'PAG',
       'THR', 'CAR', 'CINF', 'PH', 'STBA', 'MMS', 'CRUS', 'AXP', 'ALGN',
       'ON', 'SFBS'], dtype=object), 'r_CAGR/retnStd': array(['TSLA', 'CTLT', 'STNG', 'TNK', 'ASC', 'ELF', 'TNP', 'FCF', 'CAR',
       'PAG', 'ALGN', 'PAR', 'AN', 'THR', 'SMG', 'META', 'ON', 'CRUS',
       'MPWR', 'PRO'], dtype=object), 'r_retnStd/UI': array(['AXP', 'MMS', 'CTLT', 'PH', 'SFBS', 'CINF', 'ELF', 'FCF', 'STBA',
       'URI', 'DXCM', 'FISV', 'ENS', 'THR', 'STNG', 'CRUS', 'TNK', 'WBS',
       'FTSM', 'GM'], dtype=object)}}
most_freq_syms: [('CTLT', 3), ('STNG', 3), ('TNK', 3), ('ELF', 3), ('FCF', 3), ('THR', 3), ('CRUS', 3), ('TSLA', 2), ('ASC', 2), ('TNP', 2), ('PAG', 2), ('CAR', 2), ('CINF', 2), ('PH', 2), ('STBA', 2), ('MMS', 2), ('AXP', 2), ('ALGN', 2), ('ON', 2), ('SFBS', 2), ('PAR', 1), ('AN', 1), ('SMG', 1), ('META', 1), ('MPWR', 1), ('PRO', 1), ('URI', 1), ('DXCM', 1), ('FISV', 1), ('

perf_ranks: {'period-30': {'r_CAGR/UI': array(['TSLA', 'ACLS', 'CRUS', 'ICPT', 'RCL', 'NVDA', 'RVNC', 'CTLT',
       'MARA', 'MPWR', 'DCP', 'SMG', 'NATI', 'FORM', 'EVRI', 'CAR', 'ON',
       'CRS', 'MEOH', 'STX'], dtype=object), 'r_CAGR/retnStd': array(['TSLA', 'MARA', 'ACLS', 'ICPT', 'NVDA', 'RCL', 'RVNC', 'CRUS',
       'SMG', 'WBD', 'CTLT', 'MPWR', 'NCLH', 'ALGN', 'FORM', 'CAR', 'CRS',
       'PAR', 'TRIP', 'STM'], dtype=object), 'r_retnStd/UI': array(['DCP', 'CRUS', 'NATI', 'FTSM', 'CTLT', 'LRN', 'FISV', 'EVRI',
       'SWKS', 'BYD', 'EFR', 'MGM', 'ON', 'URI', 'TSLA', 'BGX', 'RVNC',
       'STX', 'WYNN', 'MEOH'], dtype=object)}}
most_freq_syms: [('TSLA', 3), ('CRUS', 3), ('RVNC', 3), ('CTLT', 3), ('ACLS', 2), ('ICPT', 2), ('RCL', 2), ('NVDA', 2), ('MARA', 2), ('MPWR', 2), ('DCP', 2), ('SMG', 2), ('NATI', 2), ('FORM', 2), ('EVRI', 2), ('CAR', 2), ('ON', 2), ('CRS', 2), ('MEOH', 2), ('STX', 2), ('WBD', 1), ('NCLH', 1), ('ALGN', 1), ('PAR', 1), ('TRIP', 1), ('STM', 1), ('FTSM', 1), ('

perf_ranks: {'period-120': {'r_CAGR/UI': array(['FTSM', 'ELF', 'HZNP', 'ACGL', 'FIVE', 'AGYS', 'ASC', 'FTI', 'RNR',
       'OI', 'RMBS', 'PERI', 'LOCO', 'NFLX', 'TPX', 'GILD', 'RCL', 'ATKR',
       'COLL', 'TTC'], dtype=object), 'r_CAGR/retnStd': array(['FTSM', 'ELF', 'ASC', 'RMBS', 'RNR', 'HZNP', 'ACLS', 'RCL', 'WYNN',
       'TEX', 'PERI', 'AXON', 'PEN', 'FTI', 'ATKR', 'AGYS', 'ACGL', 'OMC',
       'STLD', 'URI'], dtype=object), 'r_retnStd/UI': array(['FTSM', 'HZNP', 'ELF', 'BIIB', 'FIVE', 'ACGL', 'REGN', 'ISEE',
       'AGYS', 'NFLX', 'LOCO', 'IRDM', 'OI', 'COLL', 'CNO', 'TPX', 'GILD',
       'FTI', 'PCG', 'MGI'], dtype=object)}}
most_freq_syms: [('FTSM', 3), ('ELF', 3), ('HZNP', 3), ('ACGL', 3), ('AGYS', 3), ('FTI', 3), ('FIVE', 2), ('ASC', 2), ('RNR', 2), ('OI', 2), ('RMBS', 2), ('PERI', 2), ('LOCO', 2), ('NFLX', 2), ('TPX', 2), ('GILD', 2), ('RCL', 2), ('ATKR', 2), ('COLL', 2), ('TTC', 1), ('ACLS', 1), ('WYNN', 1), ('TEX', 1), ('AXON', 1), ('PEN', 1), ('OMC', 1), ('STLD', 1), ('U

perf_ranks: {'period-60': {'r_CAGR/UI': array(['FTSM', 'BVH', 'ALGN', 'BURL', 'ACLS', 'BMA', 'META', 'ANF', 'MTH',
       'HSBC', 'HZNP', 'MCFT', 'NSTG', 'PEN', 'MMS', 'SPR', 'WYNN', 'CCU',
       'TOL', 'EXAS'], dtype=object), 'r_CAGR/retnStd': array(['ACLS', 'BMA', 'BVH', 'FTSM', 'ANF', 'ALGN', 'CTLT', 'HSBC',
       'META', 'BURL', 'MTH', 'CCU', 'PAR', 'CRUS', 'NSTG', 'IBP', 'THR',
       'WYNN', 'PEN', 'NVO'], dtype=object), 'r_retnStd/UI': array(['FTSM', 'HZNP', 'MMS', 'BURL', 'AJRD', 'MCFT', 'ALGN', 'BVH',
       'EXAS', 'LW', 'META', 'TMHC', 'VRNS', 'PNM', 'MTH', 'URI', 'SPR',
       'TOL', 'PEN', 'NSTG'], dtype=object)}}
most_freq_syms: [('FTSM', 3), ('BVH', 3), ('ALGN', 3), ('BURL', 3), ('META', 3), ('MTH', 3), ('NSTG', 3), ('PEN', 3), ('ACLS', 2), ('BMA', 2), ('ANF', 2), ('HSBC', 2), ('HZNP', 2), ('MCFT', 2), ('MMS', 2), ('SPR', 2), ('WYNN', 2), ('CCU', 2), ('TOL', 2), ('EXAS', 2), ('CTLT', 1), ('PAR', 1), ('CRUS', 1), ('IBP', 1), ('THR', 1), ('NVO', 1), ('AJRD', 1), ('LW', 1

...

perf_ranks: {'period-60': {'r_CAGR/UI': array(['BVH', 'FTSM', 'META', 'ZTO', 'BURL', 'MTH', 'RAMP', 'GDS', 'WB',
       'PHM', 'PUK', 'BIDU', 'ENV', 'EXAS', 'VRNS', 'ALGN', 'FIVN', 'TOL',
       'ACLS', 'W'], dtype=object), 'r_CAGR/retnStd': array(['GDS', 'META', 'ALGN', 'W', 'ACLS', 'BVH', 'FIVN', 'RAMP', 'WB',
       'BURL', 'BIDU', 'EXAS', 'MTH', 'PUK', 'ZTO', 'AMBA', 'TPX', 'PHM',
       'VRNS', 'ANF'], dtype=object), 'r_retnStd/UI': array(['FTSM', 'BVH', 'ZTO', 'HZNP', 'BURL', 'MTH', 'TOL', 'COLL', 'TMHC',
       'PHM', 'DHI', 'MMS', 'ENV', 'PNM', 'AEM', 'NKE', 'MKSI', 'RAMP',
       'META', 'VRNS'], dtype=object)}}
most_freq_syms: [('BVH', 3), ('META', 3), ('ZTO', 3), ('BURL', 3), ('MTH', 3), ('RAMP', 3), ('PHM', 3), ('VRNS', 3), ('FTSM', 2), ('GDS', 2), ('WB', 2), ('PUK', 2), ('BIDU', 2), ('ENV', 2), ('EXAS', 2), ('ALGN', 2), ('FIVN', 2), ('TOL', 2), ('ACLS', 2), ('W', 2), ('AMBA', 1), ('TPX', 1), ('ANF', 1), ('HZNP', 1), ('COLL', 1), ('TMHC', 1), ('DHI', 1), ('MMS', 1), ('PNM',

perf_ranks: {'period-30': {'r_CAGR/UI': array(['BMA', 'BVH', 'ALGN', 'CCU', 'ACIW', 'BGNE', 'HLF', 'MARA', 'COLL',
       'HZNP', 'EXAS', 'NTES', 'FTSM', 'CROX', 'PAR', 'VLO', 'CCJ',
       'HSBC', 'AVXL', 'AGYS'], dtype=object), 'r_CAGR/retnStd': array(['BMA', 'MARA', 'BVH', 'CCU', 'ALGN', 'BGNE', 'SHYF', 'NEOG',
       'CYRX', 'PBF', 'PAR', 'AVXL', 'HLF', 'EXAS', 'NATI', 'ACIW',
       'ACLS', 'HSBC', 'GE', 'SBH'], dtype=object), 'r_retnStd/UI': array(['HZNP', 'BVH', 'ACIW', 'FTSM', 'COLL', 'ALGN', 'JOE', 'BMA',
       'AJRD', 'HLF', 'NTES', 'SGRY', 'CCJ', 'IRDM', 'CROX', 'CCU',
       'PRGO', 'WB', 'BGNE', 'DCP'], dtype=object)}}
most_freq_syms: [('BMA', 3), ('BVH', 3), ('ALGN', 3), ('CCU', 3), ('ACIW', 3), ('BGNE', 3), ('HLF', 3), ('MARA', 2), ('COLL', 2), ('HZNP', 2), ('EXAS', 2), ('NTES', 2), ('FTSM', 2), ('CROX', 2), ('PAR', 2), ('CCJ', 2), ('HSBC', 2), ('AVXL', 2), ('VLO', 1), ('AGYS', 1), ('SHYF', 1), ('NEOG', 1), ('CYRX', 1), ('PBF', 1), ('NATI', 1), ('ACLS', 1), ('GE', 1), (

perf_ranks: {'period-15': {'r_CAGR/UI': array(['TSLA', 'META', 'CTLT', 'THR', 'W', 'ICPT', 'ELF', 'ALGN', 'CRUS',
       'LRN', 'JWN', 'WBS', 'SMG', 'PNFP', 'AX', 'PACW', 'ON', 'KEX',
       'SHOP', 'ALLY'], dtype=object), 'r_CAGR/retnStd': array(['TSLA', 'W', 'CTLT', 'META', 'ICPT', 'ALGN', 'ELF', 'JWN', 'SMG',
       'SHOP', 'ON', 'THR', 'PACW', 'DBRG', 'CRUS', 'NVDA', 'LRN', 'CACC',
       'SIVB', 'WBS'], dtype=object), 'r_retnStd/UI': array(['THR', 'META', 'CRUS', 'LRN', 'PNFP', 'ELF', 'KEX', 'LOCO', 'WBS',
       'AXP', 'ICPT', 'CUBE', 'AX', 'WIRE', 'SUN', 'PERI', 'MWA', 'CINF',
       'HBAN', 'CMA'], dtype=object)}}
most_freq_syms: [('META', 3), ('THR', 3), ('ICPT', 3), ('ELF', 3), ('CRUS', 3), ('LRN', 3), ('WBS', 3), ('TSLA', 2), ('CTLT', 2), ('W', 2), ('ALGN', 2), ('JWN', 2), ('SMG', 2), ('PNFP', 2), ('AX', 2), ('PACW', 2), ('ON', 2), ('KEX', 2), ('SHOP', 2), ('ALLY', 1), ('DBRG', 1), ('NVDA', 1), ('CACC', 1), ('SIVB', 1), ('LOCO', 1), ('AXP', 1), ('CUBE', 1), ('WIRE', 1), ('SU

perf_ranks: {'period-120': {'r_CAGR/UI': array(['FTSM', 'ELF', 'HZNP', 'AGYS', 'COLL', 'FTI', 'RNR', 'LOCO', 'OI',
       'SLB', 'ACGL', 'RCL', 'BKE', 'ASC', 'PERI', 'MODN', 'GILD', 'XPRO',
       'DCP', 'RMBS'], dtype=object), 'r_CAGR/retnStd': array(['FTSM', 'ELF', 'HZNP', 'RNR', 'RMBS', 'FTI', 'ASC', 'RCL', 'ACLS',
       'AGYS', 'PEN', 'ATKR', 'PERI', 'COLL', 'XPRO', 'BKE', 'LOCO',
       'SLB', 'LVS', 'BMA'], dtype=object), 'r_retnStd/UI': array(['FTSM', 'HZNP', 'ELF', 'BIIB', 'DCP', 'IRDM', 'ACGL', 'COLL',
       'ISEE', 'AGYS', 'FIVE', 'MGI', 'REGN', 'PCG', 'LOCO', 'OI', 'NFLX',
       'TS', 'FTI', 'MODN'], dtype=object)}}
most_freq_syms: [('FTSM', 3), ('ELF', 3), ('HZNP', 3), ('AGYS', 3), ('COLL', 3), ('FTI', 3), ('LOCO', 3), ('RNR', 2), ('OI', 2), ('SLB', 2), ('ACGL', 2), ('RCL', 2), ('BKE', 2), ('ASC', 2), ('PERI', 2), ('MODN', 2), ('XPRO', 2), ('DCP', 2), ('RMBS', 2), ('GILD', 1), ('ACLS', 1), ('PEN', 1), ('ATKR', 1), ('LVS', 1), ('BMA', 1), ('BIIB', 1), ('IRDM', 1), ('ISEE'

perf_ranks: {'period-60': {'r_CAGR/UI': array(['FTSM', 'BVH', 'ALGN', 'ACLS', 'BMA', 'BURL', 'META', 'ANF', 'MTH',
       'WYNN', 'MMS', 'PEN', 'MCFT', 'HSBC', 'NSTG', 'BIDU', 'HZNP',
       'CCU', 'IPGP', 'URI'], dtype=object), 'r_CAGR/retnStd': array(['ACLS', 'BMA', 'BVH', 'ALGN', 'FTSM', 'ANF', 'CCU', 'HSBC', 'CTLT',
       'IBP', 'WYNN', 'THR', 'IPGP', 'MTH', 'META', 'NSTG', 'TEAM', 'PEN',
       'BIDU', 'TPX'], dtype=object), 'r_retnStd/UI': array(['FTSM', 'HZNP', 'MMS', 'BURL', 'AJRD', 'MCFT', 'ALGN', 'BVH', 'LW',
       'EXAS', 'META', 'WB', 'TMHC', 'VRNS', 'URI', 'MTH', 'SPR', 'PEN',
       'LVS', 'PNM'], dtype=object)}}
most_freq_syms: [('FTSM', 3), ('BVH', 3), ('ALGN', 3), ('META', 3), ('MTH', 3), ('PEN', 3), ('ACLS', 2), ('BMA', 2), ('BURL', 2), ('ANF', 2), ('WYNN', 2), ('MMS', 2), ('MCFT', 2), ('HSBC', 2), ('NSTG', 2), ('BIDU', 2), ('HZNP', 2), ('CCU', 2), ('IPGP', 2), ('URI', 2), ('CTLT', 1), ('IBP', 1), ('THR', 1), ('TEAM', 1), ('TPX', 1), ('AJRD', 1), ('LW', 1), ('EXAS',

In [12]:
def top_set_sym_freq_cnt(top_set_syms_n_freq):
    # accommodate upto 5 periods of days_lookbacks(i.e. days_lookbacks = [5, 10, 15, 20, 25])
    sym_freq_cnt_15 = []
    sym_freq_cnt_14 = []
    sym_freq_cnt_13 = []
    sym_freq_cnt_12 = []
    sym_freq_cnt_11 = []
    sym_freq_cnt_10 = []
    sym_freq_cnt_9 = []
    sym_freq_cnt_8 = []
    sym_freq_cnt_7 = []
    sym_freq_cnt_6 = []
    sym_freq_cnt_5 = []
    sym_freq_cnt_4 = []
    sym_freq_cnt_3 = []
    sym_freq_cnt_2 = []

    for sym_n_freq in top_set_syms_n_freq:
        _sym = sym_n_freq[0]
        _freq = sym_n_freq[1]
        # print(_sym, _freq)
        if _freq == 15:
            sym_freq_cnt_15.append(_sym)
        elif _freq == 14:
            sym_freq_cnt_14.append(_sym)
        elif _freq == 13:
            sym_freq_cnt_13.append(_sym)
        elif _freq == 12:
            sym_freq_cnt_12.append(_sym)                        
        elif _freq == 11:
            sym_freq_cnt_11.append(_sym)
        elif _freq == 10:
            sym_freq_cnt_10.append(_sym)            
        elif _freq == 9:
            sym_freq_cnt_9.append(_sym)
        elif _freq == 8:
            sym_freq_cnt_8.append(_sym)
        elif _freq == 7:
            sym_freq_cnt_7.append(_sym)  
        elif _freq == 6:
            sym_freq_cnt_6.append(_sym)
        elif _freq == 5:
            sym_freq_cnt_5.append(_sym)
        elif _freq == 4:
            sym_freq_cnt_4.append(_sym)
        elif _freq == 3:
            sym_freq_cnt_3.append(_sym)          
        else:
            sym_freq_cnt_2.append(_sym)

    l_sym_freq_cnt = []

    l_sym_freq_cnt.append(sym_freq_cnt_15)
    l_sym_freq_cnt.append(sym_freq_cnt_14)
    l_sym_freq_cnt.append(sym_freq_cnt_13)
    l_sym_freq_cnt.append(sym_freq_cnt_12)    
    l_sym_freq_cnt.append(sym_freq_cnt_11)   
    l_sym_freq_cnt.append(sym_freq_cnt_10)
    l_sym_freq_cnt.append(sym_freq_cnt_9)
    l_sym_freq_cnt.append(sym_freq_cnt_8)
    l_sym_freq_cnt.append(sym_freq_cnt_7)    
    l_sym_freq_cnt.append(sym_freq_cnt_6)
    l_sym_freq_cnt.append(sym_freq_cnt_5)
    l_sym_freq_cnt.append(sym_freq_cnt_4)
    l_sym_freq_cnt.append(sym_freq_cnt_3)    
    l_sym_freq_cnt.append(sym_freq_cnt_2)    

    return l_sym_freq_cnt    

In [13]:
for top_set_syms_n_freq in grp_top_set_syms_n_freq:
  l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
  if verbose:
    print(f'sym_freq_15: {l_sym_freq_cnt[0]}')
    print(f'sym_freq_14: {l_sym_freq_cnt[1]}')
    print(f'sym_freq_13: {l_sym_freq_cnt[2]}')
    print(f'sym_freq_12: {l_sym_freq_cnt[3]}')
    print(f'sym_freq_11: {l_sym_freq_cnt[4]}')
    print(f'sym_freq_10: {l_sym_freq_cnt[5]}')
    print(f'sym_freq_9: {l_sym_freq_cnt[6]}')
    print(f'sym_freq_8: {l_sym_freq_cnt[7]}')
    print(f'sym_freq_7: {l_sym_freq_cnt[8]}')
    print(f'sym_freq_6: {l_sym_freq_cnt[9]}')
    print(f'sym_freq_5: {l_sym_freq_cnt[10]}')
    print(f'sym_freq_4: {l_sym_freq_cnt[11]}')
    print(f'sym_freq_3: {l_sym_freq_cnt[12]}')
    print(f'sym_freq_2: {l_sym_freq_cnt[13]}\n')  

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: []
sym_freq_8: ['FTSM']
sym_freq_7: ['CRUS', 'CTLT']
sym_freq_6: ['ALGN', 'ELF']
sym_freq_5: ['ACLS', 'HZNP', 'TSLA']
sym_freq_4: ['ASC', 'CAR', 'META', 'MMS', 'ON', 'PEN', 'RCL', 'THR', 'URI', 'WYNN']
sym_freq_3: ['ACGL', 'AGYS']
sym_freq_2: []

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: []
sym_freq_8: ['FTSM']
sym_freq_7: ['CRUS', 'CTLT']
sym_freq_6: ['ALGN', 'ELF']
sym_freq_5: ['ACLS', 'HZNP', 'TSLA']
sym_freq_4: ['ASC', 'CAR', 'META', 'MMS', 'ON', 'PEN', 'RCL', 'THR', 'URI', 'WYNN']
sym_freq_3: ['ACGL', 'AGYS']
sym_freq_2: []

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: ['FTSM']
sym_freq_9: ['SGEN']
sym_freq_8: ['TA']
sym_freq_7: ['ACGL', 'OEC']
sym_freq_6: ['ACLS', 'ASC', 'ELF', 'LNTH', 'TNK']
sym_freq_5: ['THR', 'TNP']
sym_freq_4: ['FTI', 'HEES', 'MMS', 'NVGS', 

In [14]:
from myUtils import symb_perf_stats_vectorized_v8

print('z_grp_top_set_syms_n_freq:')
z_grp_top_set_syms_n_freq = zip(max_lookback_slices, grp_top_set_syms_n_freq)

for i, (_lookback_slice, _top_set_syms_n_freq) in enumerate(z_grp_top_set_syms_n_freq):
  iloc_start_train = _lookback_slice[0]
  iloc_end_train = _lookback_slice[1]
  iloc_start_eval = iloc_end_train
  iloc_end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_iloc_start_train = df.index[iloc_start_train].strftime('%Y-%m-%d')
  d_iloc_end_train = df.index[iloc_end_train].strftime('%Y-%m-%d')
  d_iloc_start_eval = d_iloc_end_train
  d_iloc_end_eval = df.index[iloc_end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_iloc_end_train
  d_df_eval_end = df.index[iloc_end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_iloc_start_train}, {d_iloc_end_train}, {d_iloc_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms_n_freq: {_top_set_syms_n_freq}\n')

  l_sym_freq_cnt = top_set_sym_freq_cnt(_top_set_syms_n_freq)
  if verbose:
    print(f'sym_freq_15: {l_sym_freq_cnt[0]}')
    print(f'sym_freq_14: {l_sym_freq_cnt[1]}')
    print(f'sym_freq_13: {l_sym_freq_cnt[2]}')
    print(f'sym_freq_12: {l_sym_freq_cnt[3]}')
    print(f'sym_freq_11: {l_sym_freq_cnt[4]}')
    print(f'sym_freq_10: {l_sym_freq_cnt[5]}')
    print(f'sym_freq_9: {l_sym_freq_cnt[6]}')
    print(f'sym_freq_8: {l_sym_freq_cnt[7]}')
    print(f'sym_freq_7: {l_sym_freq_cnt[8]}')
    print(f'sym_freq_6: {l_sym_freq_cnt[9]}')
    print(f'sym_freq_5: {l_sym_freq_cnt[10]}')
    print(f'sym_freq_4: {l_sym_freq_cnt[11]}')
    print(f'sym_freq_3: {l_sym_freq_cnt[12]}')
    print(f'sym_freq_2: {l_sym_freq_cnt[13]}\n')  


  _sym_idx = ['SPY']
  df_SPY = df[iloc_start_eval:iloc_end_eval][_sym_idx]

  (
    _symbols,
    _period_yr,
    _retn,
    _DD,
    _UI,
    _MDD,
    _retnMean,
    _retnStd,
    _retnStd_div_UI,
    _CAGR,
    _CAGR_div_retnStd,
    _CAGR_div_UI,
    SPY_retnStd_d_UI,     
    SPY_CAGR,
    SPY_CAGR_d_retnStd,
    SPY_CAGR_d_UI,
  ) = symb_perf_stats_vectorized_v8(df_SPY)  

  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR: {SPY_retnStd_d_UI[0]:>17,.1f}, {SPY_CAGR_d_retnStd[0]:>17,.1f}, {SPY_CAGR_d_UI[0]:>17.1f}, {SPY_CAGR[0]:>17.1f}')

  # drop last list, with frequency count 2 or less, in l_sym_freq_cnt from zip
  # zip_cnt_n_syms = zip([6,5,4,3], l_sym_freq_cnt[:-1])  
  zip_cnt_n_syms = zip([15,14,13,12,11,10,9,8,7,6,5,4,3], l_sym_freq_cnt[:-1])
  for item in zip_cnt_n_syms:
    sym_freq_cnt = item[0]
    syms = item[1]
    if syms:  # iterate ONLY if there are symbols in syms
      df_eval = df[iloc_start_eval:iloc_end_eval][syms]      

      if verbose:
        print(f'iloc_start_eval: {iloc_start_eval},  date: {d_iloc_end_train}')
        print(f'iloc_end_eval:   {iloc_end_eval},  date: {d_iloc_end_eval},  df_eval last date: {d_df_eval_end}')
        print(f'frequency count of symbol(s): {sym_freq_cnt}')      
        # print(f'\ndf_eval:\n{df_eval}\n')

        syms_n_SPY = syms + ['SPY']
        df_eval_n_SPY = df[iloc_start_eval:iloc_end_eval][syms_n_SPY]   
        print(f'\ndf_eval_n_SPY:\n{df_eval_n_SPY}\n')

      (
        _symbols,
        _period_yr,
        _retn,
        _DD,
        _UI,
        _MDD,
        _retnMean,
        _retnStd,
        _retnStd_div_UI,
        _CAGR,
        _CAGR_div_retnStd,
        _CAGR_div_UI,
        grp_retnStd_d_UI,     
        grp_CAGR,
        grp_CAGR_d_retnStd,
        grp_CAGR_d_UI,
      ) = symb_perf_stats_vectorized_v8(df_eval)  
      print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>17,.1f}, {grp_retnStd_d_UI[1]  :>17,.1f}, {grp_retnStd_d_UI[2]  :>17,.1f}')
      print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>17,.1f}, {grp_CAGR_d_retnStd[1]:>17,.1f}, {grp_CAGR_d_retnStd[2]:>17,.1f}')
      print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>17,.1f}, {grp_CAGR_d_UI[1]     :>17,.1f}, {grp_CAGR_d_UI[2]     :>17,.1f}')
      print(f'grp(CAGR):         mean, std, mean/std: {grp_CAGR[0]          :>17,.1f}, {grp_CAGR[1]          :>17,.1f}, {grp_CAGR[2]          :>17,.1f}')

      if store_results:  # record results to df
        row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end, sym_freq_cnt]
        row_add1      = [grp_CAGR[0],           grp_CAGR[1],           grp_CAGR[2]]
        row_add2      = [grp_CAGR_d_UI[0],      grp_CAGR_d_UI[1],      grp_CAGR_d_UI[2]]
        row_add3      = [grp_CAGR_d_retnStd[0], grp_CAGR_d_retnStd[1], grp_CAGR_d_retnStd[2]]
        row_add4      = [grp_retnStd_d_UI[0],   grp_retnStd_d_UI[1],   grp_retnStd_d_UI[2]]
        row_add5      = [SPY_CAGR[0], SPY_CAGR_d_UI[0], SPY_CAGR_d_retnStd[0], SPY_retnStd_d_UI[0]]
        row_add_total = row_add0 + row_add1 + row_add2 + row_add3 + row_add4 + row_add5
        print(f'row_add_total: {row_add_total}')

        if run_type == 'train':
          df_eval_sym_freq_results_train.loc[len(df_eval_sym_freq_results_train)] = row_add_total
          print(f'appended row_add to df_eval_sym_freq_results_train:\n{row_add_total}\n')
        elif run_type == 'validate':
          df_eval_sym_freq_results_validate.loc[len(df_eval_sym_freq_results_validate)] = row_add_total       
          print(f'appended row_add to df_eval_sym_freq_results_validate:\n{row_add_total}\n')
        elif run_type == 'test':
          df_eval_sym_freq_results_test.loc[len(df_eval_sym_freq_results_test)] = row_add_total       
          print(f'appended row_add to df_eval_sym_freq_results_test:\n{row_add_total}\n')
        else:
          msg_stop = f"ERROR run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
          raise SystemExit(msg_stop )                             
  
  print('='*50, '\n')  

z_grp_top_set_syms_n_freq:
1 of 57 max_lookback_slice
max_lookback_slice: (15, 135, 141)
max lookback dates: 2022-08-24, 2023-02-15, 2023-02-24
df_eval dates (inclusive): 2023-02-15 - 2023-02-23
top_set_syms_n_freq: [('FTSM', 8), ('CRUS', 7), ('CTLT', 7), ('ALGN', 6), ('ELF', 6), ('ACLS', 5), ('HZNP', 5), ('TSLA', 5), ('ASC', 4), ('CAR', 4), ('META', 4), ('MMS', 4), ('ON', 4), ('PEN', 4), ('RCL', 4), ('THR', 4), ('URI', 4), ('WYNN', 4), ('ACGL', 3), ('AGYS', 3)]

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: []
sym_freq_8: ['FTSM']
sym_freq_7: ['CRUS', 'CTLT']
sym_freq_6: ['ALGN', 'ELF']
sym_freq_5: ['ACLS', 'HZNP', 'TSLA']
sym_freq_4: ['ASC', 'CAR', 'META', 'MMS', 'ON', 'PEN', 'RCL', 'THR', 'URI', 'WYNN']
sym_freq_3: ['ACGL', 'AGYS']
sym_freq_2: []


SPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR:               0.3,             -88.3,             -30.6,              -0.8
iloc_start_eval: 135,  date: 2023-02-15
iloc_end_eval: 

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 9, -0.4609960878449404, 0.0, -inf, -35.57342538404883, 0.0, -inf, -38.10126614265908, 0.0, -inf, 0.9336546783210438, 0.0, inf, 0.2904635548283454, 45.02686281133569, 27.393639837558815, 1.6436977005735598]

iloc_start_eval: 143,  date: 2023-02-28
iloc_end_eval:   149,  date: 2023-03-08,  df_eval last date: 2023-03-07
frequency count of symbol(s): 8

df_eval_n_SPY:
                   TA         SPY
Date                             
2023-02-28  84.349998  396.260010
2023-03-01  84.260002  394.739990
2023-03-02  84.550003  397.809998
2023-03-03  84.769997  404.190002
2023-03-06  84.629997  404.470001
2023-03-07  84.510002  398.269989

grp(retnStd/UI):   mean, std, mean/std:               1.5,               0.0,               inf
grp(CAGR/retnStd): mean, std, mean/std:              46.0,               0.0,               inf
grp(CAGR/UI):      mean, std, mean/std:              67.4,               0.0,

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 6, 0.2324845602068802, 0.6768553278539498, 0.34347747685461844, 40.56683484671176, 60.19922637335775, 0.6738763484287124, 14.629432273857605, 30.986364488901437, 0.4721248366873601, 1.704632897859896, 0.5043838983201376, 3.3796338533748114, -0.09193157399702889, -10.57673291343087, -11.086939152840857, 0.9539813259208467]

iloc_start_eval: 132,  date: 2023-02-10
iloc_end_eval:   138,  date: 2023-02-21,  df_eval last date: 2023-02-17
frequency count of symbol(s): 5

df_eval_n_SPY:
                 FORM        HZNP        RCL        THR         SPY
Date                                                               
2023-02-10  31.480000  109.419998  71.639999  27.010000  408.040009
2023-02-13  31.900000  109.510002  73.989998  27.180000  412.829987
2023-02-14  32.270000  109.550003  75.370003  26.700001  412.640015
2023-02-15  32.590000  109.680000  75.750000  27.309999  413.980011
2023-02-16  32.3

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 4, 1.0315053747914182, 1.7567201748499237, 0.5871768250623862, 179.727344382073, 249.14398911198768, 0.7213794120527122, 60.1248786608953, 135.98981118500717, 0.44212781926065386, 1.393202341742599, 0.7206964742657268, 1.9331332835533113, 1.332474412927949, 389.6182726311638, 175.29048936700116, 2.222700581407073]


7 of 57 max_lookback_slice
max_lookback_slice: (11, 131, 137)
max lookback dates: 2022-08-18, 2023-02-09, 2023-02-17
df_eval dates (inclusive): 2023-02-09 - 2023-02-16
top_set_syms_n_freq: [('META', 9), ('CRUS', 7), ('ELF', 7), ('FTSM', 7), ('RCL', 6), ('THR', 6), ('ALGN', 5), ('CTLT', 5), ('HZNP', 5), ('ACLS', 4), ('CCU', 4), ('ICPT', 4), ('LVS', 4), ('NVDA', 4), ('OI', 4), ('SMG', 4), ('TSLA', 4), ('W', 4), ('AGYS', 3), ('BIDU', 3)]

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: ['META']
sym_freq_8: []
sym_freq_7: ['CRUS'

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 3, 1.8283941491210736, 1.4527238460103093, 1.2585971890958403, 1814.522494796332, 1800.3645144243546, 1.0078639521377726, 228.32754865103286, 218.91561041521126, 1.0429934540436392, 4.793428226924121, 3.289170547236809, 1.45733647984627, 0.15848600841606086, 28.17913943147191, 19.17204723250163, 1.4698033595338167]


8 of 57 max_lookback_slice
max_lookback_slice: (17, 137, 143)
max lookback dates: 2022-08-26, 2023-02-17, 2023-02-28
df_eval dates (inclusive): 2023-02-17 - 2023-02-27
top_set_syms_n_freq: [('CTLT', 7), ('FTSM', 7), ('TA', 7), ('ELF', 6), ('MMS', 6), ('THR', 6), ('ACLS', 5), ('ANF', 5), ('CRUS', 5), ('HZNP', 5), ('TWLO', 5), ('WIRE', 5), ('ASC', 4), ('CAR', 4), ('MCFT', 4), ('SMG', 4), ('TNK', 4), ('AGYS', 3), ('ALGN', 3), ('BVH', 3)]

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: []
sym_freq_8: []
sym_freq_7: ['CTLT', 'FT

...

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 6, -0.2304778960973984, 0.8855680803465115, -0.26025994072326475, -10.806578966880215, 33.45061528338456, -0.3230606933633299, -43.02966738659296, 61.777547725942654, -0.6965259867141542, 0.646137643484497, 0.2773052133310091, 2.330059488327131, 2.156044052723052, 267.065195525531, 199.46150450164606, 1.338931019260045]

iloc_start_eval: 123,  date: 2023-01-30
iloc_end_eval:   129,  date: 2023-02-07,  df_eval last date: 2023-02-06
frequency count of symbol(s): 5

df_eval_n_SPY:
                  BVH        LVS         MELI         SPY
Date                                                     
2023-01-30  31.050610  57.880001  1161.800049  400.589996
2023-01-31  32.243332  59.000000  1181.689941  406.480011
2023-02-01  32.879456  58.919998  1214.290039  410.799988
2023-02-02  33.893269  57.400002  1226.579956  416.779999
2023-02-03  34.350483  58.080002  1187.250000  412.350006
2023-02-06  32.59121

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 7, -0.273998027498982, 0.47850008247745984, -0.5726185585597865, 201.96486408083288, 214.40986047129942, 0.9419569773371854, 51.958543287123526, 90.69863014170974, 0.5728702099022028, 1.8739146757291927, 1.1597903415676276, 1.6157357140914976, -0.6968089470664469, -34.951374301004456, -73.54228274716586, 0.4752554992230153]

iloc_start_eval: 137,  date: 2023-02-17
iloc_end_eval:   143,  date: 2023-02-28,  df_eval last date: 2023-02-27
frequency count of symbol(s): 6

df_eval_n_SPY:
                  ELF        MMS        THR         SPY
Date                                                   
2023-02-17  73.650002  84.000000  26.930000  407.260010
2023-02-21  73.190002  83.269997  26.200001  399.089996
2023-02-22  73.120003  83.040001  26.480000  398.540009
2023-02-23  73.750000  82.510002  26.490000  400.660004
2023-02-24  73.339996  82.830002  26.440001  396.380005
2023-02-27  74.070000  82.2300

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 6, -0.46823393961519955, 0.0, -inf, -25.98420064600907, 0.0, -inf, -41.70112419414448, 0.0, -inf, 0.6231055192909567, 0.0, inf, 1.9111713234341527, 232.44723089034463, 181.6320410135074, 1.2797699656585277]

iloc_start_eval: 124,  date: 2023-01-31
iloc_end_eval:   130,  date: 2023-02-08,  df_eval last date: 2023-02-07
frequency count of symbol(s): 5

df_eval_n_SPY:
                  PUK         TX         SPY
Date                                        
2023-01-31  33.349998  40.349998  406.480011
2023-02-01  33.779999  40.779999  410.799988
2023-02-02  33.189999  40.160000  416.779999
2023-02-03  32.590000  39.910000  412.350006
2023-02-06  31.400000  39.230000  409.829987
2023-02-07  31.750000  38.959999  415.190002

grp(retnStd/UI):   mean, std, mean/std:               0.4,               0.0,               9.6
grp(CAGR/retnStd): mean, std, mean/std:             -66.5,              18.1,       

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 5, 1.8511685503127406, 3.417017526385, 0.5417497967214603, 67.61594104793242, 121.75988341916677, 0.5553219923442264, 56.41093806302528, 117.89698934377607, 0.47847649356453464, 0.8152090070922009, 0.18809287338955408, 4.334077056730605, 2.6644209788355644, 520.1691474488628, 272.883148241329, 1.9061973991477188]

iloc_start_eval: 120,  date: 2023-01-25
iloc_end_eval:   126,  date: 2023-02-02,  df_eval last date: 2023-02-01
frequency count of symbol(s): 4

df_eval_n_SPY:
                 ACGL       AGYS         AU    CCU       EXAS  MARA       NATI       NEOG       NTES         SPY
Date                                                                                                            
2023-01-25  64.800003  82.000000  21.920000  15.45  64.250000  8.75  54.062160  19.950001  90.300003  400.350006
2023-01-26  64.750000  84.570000  21.840000  15.54  65.040001  8.75  54.161633  19.879999  92.

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 7, 0.05291926182130524, 0.0, inf, 129625.18902694463, 0.0, inf, 330.22412977603403, 0.0, inf, 392.53699938541604, 0.0, inf, 2.6644209788355644, 520.1691474488628, 272.883148241329, 1.9061973991477188]

iloc_start_eval: 120,  date: 2023-01-25
iloc_end_eval:   126,  date: 2023-02-02,  df_eval last date: 2023-02-01
frequency count of symbol(s): 6

df_eval_n_SPY:
                  BVH       COLL        HZNP         SPY
Date                                                    
2023-01-25  30.662973  28.580000  113.000000  400.350006
2023-01-26  30.484064  28.469999  111.250000  404.750000
2023-01-27  30.951214  28.290001  110.629997  405.679993
2023-01-30  31.050610  27.379999  110.050003  400.589996
2023-01-31  32.243332  28.080000  109.720001  406.480011
2023-02-01  32.879456  29.320000  110.449997  410.799988

grp(retnStd/UI):   mean, std, mean/std:               2.7,               2.6,             

In [15]:
if store_results:  # record results to df
  if run_type == 'train':
    pickle_dump(df_eval_sym_freq_results_train, path_data_dump, fp_df_eval_sym_freq_results_train)
    print(f'Save results to: {fp_df_eval_sym_freq_results_train}')
  elif run_type == 'validate':
    pickle_dump(df_eval_sym_freq_results_validate, path_data_dump, fp_df_eval_sym_freq_results_validate)   
    print(f'Save results to: {fp_df_eval_sym_freq_results_validate}')
  elif run_type == 'test':
    pickle_dump(df_eval_sym_freq_results_test, path_data_dump, fp_df_eval_sym_freq_results_test)   
    print(f'Save results to: {fp_df_eval_sym_freq_results_test}')
  else:
    msg_stop = f"ERROR: Result was not saved. run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
    raise SystemExit(msg_stop )  

Save results to: df_eval_sym_freq_results_test_2023-03-08
